# Mise en situation

Vous êtes une jeune équipe de diplômés et vous voulez créer votre propre entreprise de fintech. Vous savez que la compagnie Crystal Dawn Private Qants essaie depuis des années de développer une stratégie de gestion de portefeuille efficace. 

Malheureusement une épidémie de *Lukehead Byus* a frappé leurs analystes avant qu'ils puissent livrer leur solution. Forts de cette information, vous ne laissez pas passer cette opportunité de leur vendre votre propre stratégie.

Votre espion qui évolue au sein de Crystal Dawn et qui a miraculeusement échappé à l'épidémie a réussi à vous obtenir un entretien vendredi prochain. 

Il vous fournit les exigeances de la compagnie qui doivent impérativement être respectées sans quoi vous n'aurez aucune chance de décrocher le contrat:

- Le portefeuille doit être **long-only** et toujours comporter au moins 120 titres - $\sum (w_i\geq.0000001) \geq 120$.
- Vous ne pouvez investir que dans les 304 titres représentatifs du S&P 500 fournis avec le présent notebook.
- La stratégie doit être 'aveugle', il est interdit d'utiliser de l'information implicite liée au futur lorsque vous roulez votre stratégie sur les données historiques. (Vous risquez d'attraper le *Luckehead Byus*.)
- Vous pouvez rebalancer votre portefeuille chaque jour.

La compagnie à qui vous essayez de vendre votre stratégie va tenir des mini conférences le samedi matin ainsi que le dimanche. Assurez-vous d'y être pour récolter des informations supplémentaires sur ce à quoi elle sera attentive lors de votre présentation.

Note: *Se retirer complètement pendant une période est autorisé ($\sum w_i=0$) si le signal de retrait est produit algorithmiquement et non la conséquence d'une décision arbitraire, gare au Luckehead Byus !*

## Livrables
Pour passer l'entretien préliminaire vous devez fournir:
- Un notebook écrit en Python ou R fonctionnel avec certaines cellules de code obligatoires.
- Un fichier submission.csv contenant la pondération de votre portefeuille (le montant investi dans chaque titre) sur l'intervalle 2000-2015 avec un format correspondant au fichier `sample_submission.csv` 

Si vous passez l'entretien préliminaire, vous devrez préparer un rapide pitch de 6 minutes pour convaincre votre client potentiel de la qualité de votre stratégie. Crystal Dawn Private Quants vous demande d'absolument présenter votre stratégie en respectant le format suivant:

- Diapo 1: intuition, processus méthodologique (données, efforts pour ne pas overfitter, contraintes d'optimisation et fonction objectif)
- Diapo 2: tableau synthèse des différentes métriques, courbes de valeur ajoutée, pires rabattement, allocation sectorielle
- Diapo 3: calendrier de performance en valeur ajoutée
- Diapo 4: vue récente du portefeuille (nombre de titre, overweight,underweight…)

Vous saurez mardi soir si vous avez passé cet entretien préliminaire et êtes sélectionnés pour l'entretien final.

In [120]:
!pip install pandas seaborn matplotlib numpy statsmodels scikit-learn tqdm getFamaFrenchFactors bs4 html5lib lxml cvxopt cvxpy


In [121]:
import pandas as pd 
import numpy as np
import cvxpy as cp
import sklearn as sk

import matplotlib.pyplot as plt
from tqdm import tqdm



## Loading data

In [122]:
ROOT='./kaggle/input/dt23-test/'


In [123]:
adjusted_close = pd.read_csv(ROOT+'series/adjusted_close.csv',index_col=0)
adjusted_close.index = pd.to_datetime(adjusted_close.index)
# Uncomment the following line to test the notebook on a reduced number of 
# products that span the entire makespan
# adjusted_close = adjusted_close.dropna(axis=1).iloc[:,:20]

adjusted_close.tail()


,adjusted_close_CSCO,adjusted_close_ISRG,adjusted_close_TPR,adjusted_close_DVN,adjusted_close_MRO,adjusted_close_BA,adjusted_close_GILD,adjusted_close_MDT,adjusted_close_V,adjusted_close_A,...,adjusted_close_CI,adjusted_close_BALL,adjusted_close_JNJ,adjusted_close_DOV,adjusted_close_CRM,adjusted_close_PGR,adjusted_close_WAT,adjusted_close_BWA,adjusted_close_BLK,adjusted_close_PPL
timestamp,,,,,,,,,,,,,,,,,,,,,
1999-11-05,25.314724,NaN,NaN,12.461711,4.853413,28.456285,1.094115,25.148404,NaN,NaN,...,23.849731,1.959694,28.722943,18.832176,NaN,4.541630,22.595,3.439710,9.844385,4.417924
1999-11-04,24.128958,NaN,NaN,12.706442,4.915478,27.859031,1.098568,25.022264,NaN,NaN,...,23.090627,1.953749,28.668492,18.617589,NaN,4.452288,23.345,3.434614,9.338102,4.417924
1999-11-03,24.646007,NaN,NaN,12.624865,4.920797,27.775543,1.168869,25.022264,NaN,NaN,...,22.712518,1.987930,28.396237,18.669090,NaN,4.413589,23.250,3.434614,9.219344,4.459399
1999-11-02,25.335406,NaN,NaN,12.686863,5.075071,27.859031,1.148247,24.066253,NaN,NaN,...,22.695200,1.969106,27.906177,18.480253,NaN,4.440344,23.405,3.423573,9.219344,4.479307
1999-11-01,25.121693,NaN,NaN,12.686863,5.197426,28.661792,1.482176,23.654637,NaN,NaN,...,21.179877,1.984462,28.614041,18.025328,NaN,4.407378,25.280,3.387052,9.219344,4.489261


In [124]:
high = pd.read_csv(ROOT+'series/high.csv',index_col=0)
high.index = pd.to_datetime(high.index)
low = pd.read_csv(ROOT+'series/low.csv',index_col=0)
low.index = pd.to_datetime(low.index)
stock_names = [c.replace('adjusted_close_', '') for c in adjusted_close.columns]
combined = high.merge(low, left_index=True, right_index=True, how='inner')
combined = combined.merge(adjusted_close, left_index=True, right_index=True, how='inner')
cutoff_date = pd.to_datetime('01-01-2016')
filtered_df = combined[combined.index <= cutoff_date]

In [125]:
filtered_df = filtered_df.iloc[::-1]
for c in stock_names:
    filtered_df[f"ri_{c}"] = filtered_df[f"adjusted_close_{c}"].pct_change()
    filtered_df[f"ri_{c}"].iloc[0] = 0

for c in stock_names:
    filtered_df[f"high-low_{c}"] = filtered_df[f"high_{c}"] - filtered_df[f"low_{c}"]

for c in stock_names:
    filtered_df[f"rolling_diff_year_{c}"] = filtered_df[f"adjusted_close_{c}"].diff(periods=365)
    filtered_df[f"rolling_diff_month_{c}"] = filtered_df[f"adjusted_close_{c}"].diff(periods=30)
    filtered_df.loc[filtered_df.index[:365], f"rolling_diff_year_{c}"] = filtered_df[f"adjusted_close_{c}"] - filtered_df[f"adjusted_close_{c}"].iloc[0]
    filtered_df.loc[filtered_df.index[:30], f"rolling_diff_month_{c}"] = filtered_df[f"adjusted_close_{c}"] - filtered_df[f"adjusted_close_{c}"].iloc[0]
filtered_df.tail()

C:\Users\hamzp\AppData\Local\Temp\ipykernel_5480\623100021.py:3: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  filtered_df[f"ri_{c}"] = filtered_df[f"adjusted_close_{c}"].pct_change()
C:\Users\hamzp\AppData\Local\Temp\ipykernel_5480\623100021.py:3: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Call ffill before calling pct_change to retain current behavior and silence this warning.
  filtered_df[f"ri_{c}"] = filtered_df[f"adjusted_close_{c}"].pct_change()
C:\Users\hamzp\AppData\Local\Temp\ipykernel_5480\623100021.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-frag

,high_CSCO,high_ISRG,high_TPR,high_DVN,high_MRO,high_BA,high_GILD,high_MDT,high_V,high_A,...,rolling_diff_year_PGR,rolling_diff_month_PGR,rolling_diff_year_WAT,rolling_diff_month_WAT,rolling_diff_year_BWA,rolling_diff_month_BWA,rolling_diff_year_BLK,rolling_diff_month_BLK,rolling_diff_year_PPL,rolling_diff_month_PPL
timestamp,,,,,,,,,,,,,,,,,,,,,
2015-12-24,27.5500,549.00,33.22,33.47,14.1100,143.93,103.90,78.07,78.71,42.26,...,6.599184,0.338337,32.44,2.27,-16.139349,1.686445,19.575351,-9.019379,3.567274,0.451795
2015-12-28,27.3700,549.50,33.05,32.48,13.5700,144.67,103.31,77.50,78.65,42.08,...,6.786687,0.741120,34.75,4.55,-15.450006,2.066583,29.550248,-0.999356,3.675703,0.833845
2015-12-29,27.8200,556.60,33.23,32.84,13.4400,147.56,103.55,78.03,79.39,42.48,...,7.189469,1.401683,34.65,7.48,-15.726623,3.341364,28.066146,1.972919,3.578117,1.212331
2015-12-30,27.7700,555.63,33.02,32.53,13.0664,148.00,103.05,78.06,79.19,42.38,...,7.021759,1.119735,34.34,5.12,-15.551466,2.712221,28.566118,-6.698220,3.550823,0.479775
2015-12-31,27.4499,552.88,32.90,32.20,12.6600,145.96,102.42,77.67,78.80,42.35,...,6.642102,0.845843,27.42,2.81,-16.756463,2.112788,26.963798,-8.841054,3.483335,0.785894


In [126]:
# for c in stock_names:
#     var_mean = normalized_df[f"rolling_diff_month_{c}"].iloc[0]
#     def rolling_mean(x):
#         return x*0.05 + var_mean*0.95
#     
#     normalized_df[f"rolling_avg_diff_month_{c}"] = normalized_df[f"rolling_diff_month_{c}"].apply(rolling_mean)
# for c in stock_names:
#     var_mean = normalized_df[f"high-low_{c}"].iloc[0]
#     normalized_df[f"rolling_avg_low_high_{c}"] = normalized_df[f"high-low_{c}"].apply(rolling_mean)

In [127]:
# import getFamaFrenchFactors as gff
# factors = pd.DataFrame(gff.famaFrench3Factor(frequency='m'))
# factors.index = pd.to_datetime(factors.date_ff_factors)
# factors.index = factors.index - pd.DateOffset(months=1)

# factors_resampled = factors.resample('D').ffill()
# combined_with_factors = normalized_df.merge(factors_resampled, left_index=True, right_index=True, how='left')
# combined_with_factors.head()

In [128]:
# combined_with_factors.to_csv("normalised_with_factors.csv")

In [129]:
factors = pd.read_csv('F-F_Research_Data_Factors_daily.CSV')
factors = factors.rename(columns={'Unnamed: 0': 'Timestamp'})
factors['Timestamp'] = pd.to_datetime(factors['Timestamp'], format='%Y%m%d')
factors.set_index('Timestamp', inplace=True)
factors

,Mkt-RF,SMB,HML,RF
Timestamp,,,,
1926-07-01,0.10,-0.25,-0.27,0.009
1926-07-02,0.45,-0.33,-0.06,0.009
1926-07-06,0.17,0.30,-0.39,0.009
1926-07-07,0.09,-0.58,0.02,0.009
1926-07-08,0.21,-0.38,0.19,0.009
...,...,...,...,...
2023-08-25,0.65,-0.07,-0.58,0.020
2023-08-28,0.63,-0.01,0.41,0.020
2023-08-29,1.50,0.01,-0.11,0.020


In [130]:
from sklearn.preprocessing import MinMaxScaler

filtered_df = filtered_df.merge(factors, left_index=True, right_index=True, how='inner')
scaler = MinMaxScaler()
normalized_df = pd.DataFrame(scaler.fit_transform(filtered_df), columns=filtered_df.columns, index=filtered_df.index)
normalized_df

C:\Users\hamzp\miniconda3\envs\Desjardins-challenge\Lib\site-packages\sklearn\preprocessing\_data.py:480: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\hamzp\miniconda3\envs\Desjardins-challenge\Lib\site-packages\sklearn\preprocessing\_data.py:481: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


,high_CSCO,high_ISRG,high_TPR,high_DVN,high_MRO,high_BA,high_GILD,high_MDT,high_V,high_A,...,rolling_diff_year_BWA,rolling_diff_month_BWA,rolling_diff_year_BLK,rolling_diff_month_BLK,rolling_diff_year_PPL,rolling_diff_month_PPL,Mkt-RF,SMB,HML,RF
1999-11-01,0.470892,NaN,NaN,0.096768,0.139186,0.146316,0.364213,0.207854,NaN,NaN,...,0.446019,0.500610,0.293236,0.611430,0.491052,0.648905,0.422167,0.674144,0.461704,0.653846
1999-11-02,0.474526,NaN,NaN,0.096768,0.140257,0.136946,0.236041,0.242009,NaN,NaN,...,0.446821,0.502475,0.293236,0.611430,0.490569,0.647928,0.424631,0.561983,0.545847,0.653846
1999-11-03,0.473145,NaN,NaN,0.101882,0.131445,0.140244,0.247885,0.239817,NaN,NaN,...,0.447064,0.503039,0.293236,0.611430,0.489604,0.645973,0.474877,0.676505,0.396980,0.653846
1999-11-04,0.468203,NaN,NaN,0.095438,0.126338,0.138820,0.232022,0.244384,NaN,NaN,...,0.447064,0.503039,0.293906,0.612427,0.487593,0.641902,0.473399,0.528926,0.421791,0.653846
1999-11-05,0.467258,NaN,NaN,0.095438,0.128397,0.146316,0.200296,0.252237,NaN,NaN,...,0.447176,0.503300,0.296763,0.616677,0.487593,0.641902,0.479803,0.499410,0.444444,0.653846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-12-24,0.133295,0.922382,0.313644,0.038871,0.011942,0.888314,0.794099,0.973881,0.147232,0.206337,...,0.091423,0.586757,0.403703,0.535709,0.664044,0.693252,0.435468,0.585596,0.470334,0.000000
2015-12-28,0.131986,0.923228,0.311134,0.028744,0.007495,0.893861,0.787860,0.963470,0.146976,0.205144,...,0.106569,0.606175,0.459992,0.603040,0.669303,0.730754,0.427094,0.495868,0.432578,0.000000
2015-12-29,0.135257,0.935237,0.313792,0.032426,0.006424,0.915524,0.790398,0.973151,0.150132,0.207795,...,0.100491,0.671294,0.451617,0.627993,0.664570,0.767905,0.492611,0.563164,0.442287,0.000000
2015-12-30,0.134894,0.933596,0.310691,0.029255,0.003347,0.918822,0.785110,0.973699,0.149279,0.207132,...,0.104340,0.639156,0.454439,0.555196,0.663247,0.695999,0.404433,0.531287,0.456311,0.000000


In [131]:
# values = {}
# for stock in stock_names:
#     stock_value = {}
#     name_len = len(stock)
#     for key in normalized_df.keys():
#         if key == 'Mkt-RF' or key == 'SMB' or key == 'HML' or key == 'RF':
#             stock_value[key] = normalized_df[key]
#         elif key[-name_len:] == stock:
#             stock_value[key[:-(name_len+1)]] = normalized_df[key]
#     values[stock] = pd.DataFrame.from_dict(stock_value)
# values['CSCO'].head()

In [132]:
# test = values['CSCO'].dropna(axis=0)
# test

In [133]:


from sklearn.linear_model import Ridge
import statsmodels.api as sm

def osl_model(y, x):
    # Estime les bêtas par régression
    model = sm.OLS(y, sm.tools.add_constant(x))
    result = model.fit_regularized(method='sqrt_lasso')
    return result

# x = test[['RF', 'Mkt-RF', 'SMB', 'HML', 'rolling_avg_diff_month', 'rolling_avg_low_high']]
# y = test['ri']
#y = (y - y.mean()) / y.std()

def ridge_model(y, x):
    logistic_reg = Ridge(alpha=1.0) 
    if len(x) < 30:
        a = 2
    logistic_reg.fit(x.iloc[:-1], y.iloc[:-1])
    y_pred = logistic_reg.predict(x.tail(1))
    y_pred = pd.DataFrame(y_pred)
    y_pred.index =  [y.index.max()]
    return y_pred

In [134]:
# res = osl_model(y, x)
# coefficients = res.params

In [135]:
#y_pred = coefficients["const"] + coefficients["Mkt-RF"] * x["Mkt-RF"] + coefficients["SMB"] * x["SMB"] + coefficients["HML"] * x["HML"] + coefficients["rolling_avg_diff_month"] * x["rolling_avg_diff_month"] + coefficients["rolling_avg_low_high"] * x["rolling_avg_low_high"]
#y_pred = ridge_model(y, x)
# plt.subplots(1, 1, figsize=(12, 6))
# plt.plot(y[:100], label="y (réel)")
# plt.plot(y_pred[:100], label="y_pred (prédiction)")

# plt.legend()
# plt.show()

In [136]:
# values['ISRG']

In [137]:
def predict(df):
    pred = pd.DataFrame()
    for stock in stock_names:
        init = df[[f'RF', 'Mkt-RF', 'SMB', 'HML', f'ri_{stock}']].dropna(axis=0)
        x = init[[f'RF', f'Mkt-RF', f'SMB', f'HML']]
        y = init[f'ri_{stock}']
        if len(x) < 5:
            y_pred = pd.DataFrame([np.NAN])
        else:
            y_pred = ridge_model(y, x)
        #y_pred = coefficients["const"] + coefficients["Mkt-RF"] * x["Mkt-RF"] + coefficients["SMB"] * x["SMB"] + coefficients["HML"] * x["HML"] + coefficients["rolling_avg_diff_month"] * x["rolling_avg_diff_month"] + coefficients["rolling_avg_low_high"] * x["rolling_avg_low_high"]
        y_pred = y_pred.rename(columns={0: stock})
        pred = pd.concat([pred, y_pred], axis=1, join='outer')
    return pred

In [138]:
# pred = pred[(pred.index >= pd.to_datetime('2000-01-01')) & (pred.index <= pd.to_datetime('2016-01-01'))]
# pred

In [139]:
#pred.to_csv("predictions.csv")

In [140]:

# y = (y - y.mean()) / y.std()
# plt.subplots(1, 1, figsize=(12, 6))
# plt.plot(y, label="y (réel)")
# plt.plot(pred['CSCO'], label="y_pred (prédiction)")

# plt.legend()
# plt.show()

## Pondération du portefeuille
Entrez ici les fonctions que vous utiliserez pour générer les poids de votre portefeuille sur base d'indicateurs tirés des données historiques ou produits avec des méthodes statistiques et/ou d'apprentissage machine.

**Assurez vous que la relation $\sum_{i=1}^{i=302}w_i=1$ est toujours respectée.**

**Prenez gare aux NaNs présents dans le dataset, ils correspondent à des moments ou certains produits ne soit plus soit pas encore disponibles**

Les méthodes que vous voyez ici ne sont pas valides pour la soumission finale:
 - `rand_weights`: sélectionne aléatoirement 5 titres et génère des poids aléatoires.
 - `best_return_fibo`: sélectionne les 10 meilleurs titres en fonction d'une prédiction sur leur rendement et leur assigne des poids proportionnels aux 10 premiers termes de la suite de Fibonacci.
 - `equal_weights`: produit un portefeuille balancé de manière uniforme, reproduit le benchmark

In [141]:
file_path = ROOT+'series/additional_data/SP500_symbol_info.csv'

# Read sectors file
sectors_file = pd.read_csv(file_path)
sectors= {}
for index, row in sectors_file.iterrows():
    key = row['GICS Sector']
    value = row['Symbol']
    if key in sectors:
        sectors[key].append(value)
    else:
        sectors[key] = [value]

bmrk = pd.read_csv(ROOT+'series/benchmark/benchmark_weights.csv',index_col=0)
bmrk.index = pd.to_datetime(bmrk.index)

In [142]:
def rand_weights(returns,*args,**kwargs):
    """
        Produces random weights for num_to_select assets 
    """
    # The following array will be True for all products i that are available
    product_availability_mask = ~returns.iloc[-1].isna().values
    
    # The following array will contain all the indices i of available products
    available_product_indices = np.nonzero(product_availability_mask)[0]
    
    weights = np.zeros(len(returns.T))
    num_to_select = 125
    
    # Generate unique random indices within the array size
    random_indices = np.random.choice(available_product_indices, num_to_select, replace=False)

    # Create a boolean mask based on the randomly selected indices
    mask = np.zeros(len(weights), dtype=bool)
    mask[random_indices] = True
    rw = np.random.rand(sum(mask))
    rw /= sum(rw)
    weights[mask]=rw
    return pd.Series({"weight_"+a.split('_')[-1]:b for a,b in zip(returns.columns,weights)})

def best_returns_fibo(returns,*args,**kwargs):
    """ 
        Selects the 10 best returns and assigns weights according to a fibonacci ratio 
    """
    weights = np.zeros(len(returns.T))

    # Select the indices of the 10 best stocks
    indices=np.argsort(returns.iloc[-1].values)[::-1]
    indices=indices[~np.isnan(returns.values[0][indices])][:10]

    # Generate weights according to a fibonacci sequence
    fibo = np.array([1, 1, 2, 3, 5, 8, 13, 21, 34, 55])[::-1].astype(float)
    fibo /= sum(fibo)

    # Create a boolean mask based on the randomly selected indices
    mask = np.zeros(len(weights), dtype=bool)
    mask[indices] = True
    
    weights[mask]=fibo
    return pd.Series({"weight_"+a.split('_')[-1]:b for a,b in zip(returns.columns,weights)})

def equal_weights(returns,*args,**kwargs):
    """
        Produces an equally weighted portfolio
        this is how the benchmark is generated
    """
    weights = np.zeros(len(returns.T))
    indices = np.arange(len(weights))[~np.isnan(returns.values[0])]
    weights[indices] = 1/len(indices)
    return pd.Series({"weight_"+a.split('_')[-1]:b for a,b in zip(returns.columns,weights)})

def best_returns_weights(returns,*args,**kwargs):
    num_to_select = 120

    indices = np.argsort(returns.iloc[-1].values)[::-1]
    indices = indices[~np.isnan(returns.values[0][indices])][:num_to_select]
    weights = np.zeros(len(returns.T))
    mask = np.zeros(len(weights), dtype=bool)
    mask[indices] = True
    weights[mask] = (returns.iloc[-1][indices] / np.sum(returns.iloc[-1][indices]))
    return pd.Series({"weight_"+a.split('_')[-1]:b if not pd.isna(b) else 0.0 for a,b in zip(returns.columns,weights)})

def optimisation_weights(returns, initial_weights, *args, **kwargs):
    ROULEMENT = 0.25
    MIN_NUMBER_HOLDINGS = 120
    MIN_HOLDINGS_SIZE = 0.0005
    MAX_WEIGHT_PER_SECTOR = 0.25
    
    cols = returns.columns
    returns.fillna(-99999, inplace=True)
    returns = returns.values.tolist()[0]
    portfolio_weights = cp.Variable(len(returns))
    
    in_portfolio = cp.Variable(len(returns), boolean=True)
    nan_returns = pd.isna(returns)
    in_portfolio.value = ~nan_returns
    
    constraints = [
        cp.sum(portfolio_weights) == 1.0,
        portfolio_weights >= 0.0,
        portfolio_weights <= 1.0,
        cp.sum(in_portfolio) >= MIN_NUMBER_HOLDINGS,
    ]
    
    if not initial_weights.empty:
        initial_weights.fillna(0, inplace=True)
        initial_weights = initial_weights.values.tolist()
        constraints += [cp.sum(cp.abs(portfolio_weights - initial_weights)) / 2 <= ROULEMENT]
    eps = 1e-5
    constraints += [
        -1 + eps <= portfolio_weights - in_portfolio,
        portfolio_weights - in_portfolio <= 0,
        portfolio_weights - in_portfolio >= MIN_HOLDINGS_SIZE - 1
    ]
    
    for s, stocks in sectors.items():
        mask = [c in stocks for c in cols]
        sector_weights = cp.sum(portfolio_weights[mask])
        constraints += [sector_weights <= MAX_WEIGHT_PER_SECTOR]
    
    objective = cp.Maximize(cp.sum(returns @ portfolio_weights))
    prob = cp.Problem(objective, constraints)
    result = prob.solve(verbose=False, solver=cp.SCIPY)
    
    return pd.Series({"weight_"+a.split('_')[-1]:b if not pd.isna(b) else 0.0 for a,b in zip(cols,portfolio_weights.value)})   

## Modèle prédictif
Programmez ici vos modèles prédictifs et analytiques qui vous permettront de choisir la meilleure pondération possible étant donnée les données antérieures au moment du rebalancement.

C'est ici qu'il faut faire attention à ne pas attraper le *Lukehead Byus*, vous prenez de gros risques si les arguments de vos fonctions prennent en compte des données postérieures au moment du rebalancement ('savoir' à priori qu'AAPL va bien performer constitue une donnée postérieure). 

Le modèle actuel est très simpliste:
- `polynomial_regression`: effectue une simple régression avec un polynome du 3e degré pour essayer de prédire le rendement que fera chaque titre le lendemain.

In [143]:
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures

def polynomial_regression(past_prices):
    """
        Fits a 3d degree polynomial for each asset and returns the predicted value
    """

    models={}

    # Fitting a 3d degree polynomial for each asset
    for c in past_prices.columns:
        x=past_prices[[c]]
        xp = PolynomialFeatures(degree=3).fit_transform(np.arange(len(x)).reshape((len(x),1)))
        model = sm.OLS(x.values, xp).fit()
        models[c] = model

    # Returning the expected return for the next timestep for all stocks
    return pd.DataFrame({k:v.predict(PolynomialFeatures(degree=3).fit_transform(np.array([len(x)]).reshape((1,1)))) for k,v in models.items()})/past_prices.iloc[-1]


## Rebalancement journalier

Vous choisissez ici ce que vous fournissez aux fonctions précédemment implémentées.

L'utilisation de `rolling()` ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html)) est pratiquement obligatoire pour éviter d'incorporer les données futures dans votre système.

Si vous ne respectez pas l'intervalle `2000-01-01`-`2015-01-01` votre solution ne sera pas acceptée. 

In [144]:
# weights_df = []
# index = []

# start,stop = pd.to_datetime('2000-01-01'),pd.to_datetime('2015-01-01')

# weights = pd.Series()
# for i, x in pred.iterrows():
#     if i < start or i > stop:
#         continue
#     x = x.to_frame().T
#     weights = optimisation_weights(x, weights)
        
#     # # The following line checks that you do not invest in unavailable positions
#     assert (weights.values[np.where(x.iloc[-1].isna())]!=0.0).sum()==0
    
#     weights_df.append(weights)


In [145]:
weights_df = []
index = []

rolling_start = pd.to_datetime('1999-01-01') # Observation period starts maximum 1 year before the strategy starts (trailing_window_size<=252)
start,stop = pd.to_datetime('2000-05-01'),pd.to_datetime('2015-01-01')

trailing_window_size = 30
weights = pd.Series()
filtered_df = normalized_df[(normalized_df.index >= pd.to_datetime('1999-01-01')) & (normalized_df.index <= pd.to_datetime('2001-01-01'))]
for x in tqdm(filtered_df.rolling(trailing_window_size)):

    # If the trailing window is the right size and the current date is after the start date
    if len(x)==trailing_window_size and x.index.max()>=start:

        index.append(x.index.max())
        weights = optimisation_weights(predict(x), weights)
        
        # The following line checks that you do not invest in unavailable positions
        # assert (weights.values[np.where(x.iloc[-1].isna())]!=0.0).sum()==0
        
        weights_df.append(weights)

263it [05:55,  1.35s/it]


KeyboardInterrupt: 

#### Génération de la soumission avec le format requis. 
**Assurez vous de bien trier les dates et d'ajouter une colonne `id`.**

In [ ]:
weights_df = pd.DataFrame(weights_df,index=index)
weights_df[weights_df.index>=start].sort_index().reset_index(names='date').to_csv('submission.csv',index_label='id')

## Evaluation

#### Les cellules qui suivent doivent impérativement figurer et être fonctionnelles dans le notebook que vous fournirez sous peine de disqualification.

In [ ]:
weights_df = pd.read_csv('submission.csv',index_col=1)
weights_df.drop(columns=['id'],inplace=True)
weights_df.index = pd.to_datetime(weights_df.index)
prices_df = adjusted_close.loc[weights_df.index]
returns_df = prices_df.shift(-1)/prices_df


### Graphes

In [ ]:
bmrk = pd.read_csv(ROOT+'series/benchmark/benchmark_returns.csv',index_col=0)
bmrk.index = pd.to_datetime(bmrk.index)


In [ ]:
portfolio_performance = pd.DataFrame(pd.DataFrame(returns_df.values*weights_df.values, columns=weights_df.columns, index=weights_df.index).sum(axis=1),columns=['portfolio_return'])[:-2]
daily_returns = portfolio_performance.merge(bmrk,left_index=True,right_index=True,how='left')-1
cumulative_returns = ((daily_returns+1).cumprod()-1)
cumulative_returns.plot(figsize=(12,3))
plt.grid()


In [ ]:
(cumulative_returns['portfolio_return']-cumulative_returns['benchmark_return']).plot(figsize=(12,3),label='excess return',color='r',legend=True)
plt.title('Excess return of the strategy')
plt.grid()


In [ ]:
daily_returns.plot(figsize=(12,3))
plt.grid()


### Valeur ajoutée annualisée

In [ ]:
annualized_return = (1+daily_returns).prod()**(252/len(daily_returns))-1
added_value_annualized = annualized_return['portfolio_return']-annualized_return['benchmark_return']
added_value_annualized


### Risque actif annualisé

In [ ]:
excess_return = daily_returns['portfolio_return']-daily_returns['benchmark_return']
active_risk_annualized = excess_return.std()*np.sqrt(252)
active_risk_annualized


### Ratio d'information annualisé
C'est cette métrique qui est utilisée sur Kaggle

In [ ]:
information_ratio_annualized = added_value_annualized/active_risk_annualized
information_ratio_annualized


### Pire rabattement relatif

In [ ]:
max_seen_so_far = cumulative_returns.cummax()
drawdown = (cumulative_returns-max_seen_so_far)/max_seen_so_far
worst_drawdown = drawdown.min()
relative_worst_drawdown = worst_drawdown['portfolio_return']-worst_drawdown['benchmark_return']
relative_worst_drawdown


### Nombre de titres moyens

In [ ]:
average_number_of_stocks = (weights_df>0).sum(axis=1).mean()
average_number_of_stocks


### Taux de roulement annuel

In [ ]:
turnover_annual = (weights_df.diff().abs().sum(axis=1)/2).mean()*252
turnover_annual


## Sanity check
Ces cellules vous permettent de voir si votre solution est conforme aux contraintes mentionnées dans l'énoncé ci-dessus.

In [ ]:
for i,x in enumerate(weights_df.values):

    # Checks you respect the long-only and the cardinality constraint 
    n_positions = sum(x>.0000001)
    assert n_positions==0 or n_positions>=120

    # Checks you do not invest in non existing stocks
    assert (x[np.where(adjusted_close.loc[weights_df.index[i]].isna())]!=0.0).sum()==0
        